In [ ]:
import numpy as np
import pandas as pd
import plotnine as pn
import pyranges as pr
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from pyarrow.parquet import ParquetFile

In [ ]:
gtf_file = "/s/genomes/Gencode/Gencode_human/release_34/GRCh37_mapping/gencode.v34lift37.annotation.gtf.gz"
genome_annotation = pr.read_gtf(gtf_file)
gene_annotation = genome_annotation[genome_annotation.Feature=="gene"]

In [ ]:
import pandas as pd

# Read parquet files and convert to pandas DataFrame
dt = pd.read_parquet('/s/project/geno2pheno/predictions/bayesian/best_model_pred/v108cov_deepRVAT_genes_extended.pq')
dt_ols = pd.read_parquet('/s/project/geno2pheno/predictions/bayesian/best_model_pred/ols_filteredv3_deepRVAT_genes_extended.pq')
# dt_flat = pd.read_parquet('/s/project/geno2pheno/predictions/bayesian/v107cov_flat_deepRVAT_genes_extended.pq')

# Concatenate DataFrames
dt = pd.concat([dt[['model', 'trait', 'phenocode', 'gene_id', 'neglog_pval', 'pd', 'significant', 'replicated', 'beta']], 
                dt_ols[['model', 'trait', 'phenocode', 'gene_id', 'neglog_pval', 'significant', 'replicated', 'beta']]], 
                ignore_index=True)

# Load gene names
gene_names = pd.read_csv('/s/project/geno2pheno/data/hgnc2ensg.tsv', sep='\t')[['Ensembl gene ID', 'Approved symbol']].drop_duplicates().rename(columns={'Ensembl gene ID':'gene_id', 'Approved symbol':'gene_name'})

# Merge table with gene names
dt = dt.merge(gene_names, on='gene_id')
dt['significant'] = np.where(dt['pd']>0.999, True, False)
dt

In [ ]:
trait_gene_dict = {
    'Lymphocyte_percentage':'ARHGAP15',   # 30180
    'Platelet_crit':'RHOF',               # 30090
}

for tr in trait_gene_dict:
    print(tr, trait_gene_dict[tr])

In [ ]:
plt_genes = pd.concat([dt[(dt.trait==tr) & (dt.gene_name == trait_gene_dict[tr]) & (dt.model=='omics_pops_bayesian_v108cov_deepRVAT')] for tr in trait_gene_dict])
plt_genes

In [ ]:
nom_pval_dt = pd.concat([dt[(dt.trait==tr) & (dt.gene_name == trait_gene_dict[tr]) & (dt.model=='ols_deepRVAT_cov')] for tr in trait_gene_dict])
nom_pval_dt['nom_pval'] = np.exp(-nom_pval_dt['neglog_pval'])
# nom_pval_dt
plt_genes = pd.merge(plt_genes, nom_pval_dt[['gene_id', 'nom_pval']], on='gene_id')
plt_genes

In [ ]:

trait_cols = [f'{tr}_common_resid' for tr in trait_gene_dict]

# 200K UK Biobank
# deeprvat_gt = pd.read_parquet("/s/project/uk_biobank/processed/derived_datasets/ukbb_wes_200k_DeepRVAT_mean_6_0.parquet", columns=list(plt_genes.gene_id)).reset_index()
# deeprvat_gt = pd.melt(deeprvat_gt, id_vars='individual', value_name='deeprvat')

# plof_gt = pd.read_parquet("/s/project/uk_biobank/processed/derived_datasets/ukbb_wes_200k_ensembl_loftee_plof_binarized_pivot.parquet", columns=list(plt_genes.gene_id)).reset_index()
# plof_gt = pd.melt(plof_gt, id_vars='individual', value_name='plof')

# pheno_dt = pd.read_parquet("/s/project/uk_biobank/processed/derived_datasets/ukbb_common_traits_and_covariates_with_split_and_residuals_filteredv3.parquet", columns=['individual']+trait_cols)
# # geno_dt = deeprvat_gt.merge(plof_gt, on=['individual', 'gene']).merge(gene_names, left_on='gene', right_on='gene_id')
# plot_dt = pheno_dt.merge(deeprvat_gt.merge(plof_gt, on=['individual', 'gene']), on='individual').merge(gene_names, left_on='gene', right_on='gene_id').drop(columns='gene')
# plot_dt

# 300K UK Biobank
deeprvat_gt = pd.read_parquet("/s/project/uk_biobank/processed/derived_datasets/ukbb_wes_300k_DeepRVAT_mean_6_0.parquet", columns=list(plt_genes.gene_id)).reset_index()
deeprvat_gt = pd.melt(deeprvat_gt, id_vars='individual', value_name='deeprvat')

plof_gt = pd.read_parquet("/s/project/uk_biobank/processed/derived_datasets/ukbb_wes_500k_vep_plof_binarized.parquet", columns=list(plt_genes.gene_id)).reset_index()
plof_gt = pd.melt(plof_gt, id_vars='individual', value_name='plof')

pheno_dt = pd.read_parquet("/s/project/uk_biobank/processed/derived_datasets/ukbb_common_traits_and_covariates_with_split_and_residuals_300k_filteredv3.parquet", columns=['individual']+trait_cols)
plot_dt = pheno_dt.merge(deeprvat_gt.merge(plof_gt, on=['individual', 'gene']), on='individual').merge(gene_names, left_on='gene', right_on='gene_id').drop(columns='gene')
plot_dt

In [ ]:
#NEW
def get_vars_by_gene(var_mac_parquet_file, gene_id, gene_annotation, padding = 0, fillna = True):
    var_names = ParquetFile(var_mac_parquet_file).schema.names
    split_var_names = pd.Series(var_names[6:]).str.split(":", expand=True)
    
    variants = pr.from_dict({
        "Chromosome":split_var_names[0].astype(str), 
        "Start":split_var_names[1].astype(int), 
        "End":split_var_names[1].astype(int)+1,
        "var_name":pd.Series(var_names[6:])
    })
    
    gene_annotation_expanded = gene_annotation.copy()
    gene_annotation_expanded.Start = gene_annotation_expanded.Start - padding
    gene_annotation_expanded.End = gene_annotation_expanded.End + padding
    
    included_vars = variants.intersect(gene_annotation_expanded[gene_annotation_expanded.gene_id.str.startswith(gene_id)])
    included_vars = included_vars.as_df()["var_name"].to_list() if included_vars else []
    mac_df = pd.read_parquet(var_mac_parquet_file, columns = ["IID"]+included_vars)
    
    mac_df[included_vars] = mac_df[included_vars].fillna(mac_df[included_vars].median()).astype("Int8") if fillna else mac_df[included_vars].astype("Int8")
    return mac_df


def get_common_variant_covariates(gene_id, trait=None, phenocode=None, padding=100_000):
    if phenocode is None:
        phenocode = phenocode_dict[trait]
        
    mac_index_vars_parquet = f"/s/project/deeprvat/clumping/clumping_shubhankar/{phenocode}.0/GWAS_variants_clumped_mac.parquet" # Eva's results, filtered for Caucasians and more stringent p-val
    # mac_index_vars_parquet = f"/s/project/uk_biobank/processed/clumping/{phenocode}/GWAS_variants_clumped_mac.parquet"
    res_all = get_vars_by_gene(mac_index_vars_parquet, gene_id, gene_annotation, padding = 100_000)
    res_all['individual'] = res_all['IID'].str.split('_').str[0]
    res = res_all.drop(columns='IID')
    res = res[res.individual.isin(plot_dt.individual)]
    res.set_index('individual', inplace=True)
    res = 2 - res
    res.reset_index(inplace=True)
    
    return res

In [ ]:

lm_dt = plot_dt.dropna()

slopes = []
slope_SEs = []
pvals = []
for index, row in plt_genes.iterrows():
    if row.trait in ['Lymphocyte_percentage', 'Platelet_crit', 'glycated_haemoglobin_hba1c', 'Calcium']:
        com_covs = get_common_variant_covariates(row.gene_id, phenocode=row.phenocode)
        a = pd.merge(com_covs, lm_dt[(lm_dt.gene_name==row.gene_name)][[f'{row.trait}_common_resid', 'individual']], on='individual')
        a = a.set_index('individual')
    
        lm = LinearRegression().fit(a.drop(columns=f'{row.trait}_common_resid'), a[f'{row.trait}_common_resid'])
        prediction = lm.predict(a.drop(columns=f'{row.trait}_common_resid'))
        residual = a[f'{row.trait}_common_resid'] - prediction
        residual = residual.rename(f'{row.trait}_residual')
        
        lm_dt = pd.merge(lm_dt, residual, on='individual')
        slope, intercept, r, p, se = linregress(lm_dt[(lm_dt.gene_name==row.gene_name)]['deeprvat'], lm_dt[(lm_dt.gene_name==row.gene_name)][f'{row.trait}_residual'])

    else:
        slope, intercept, r, p, se = linregress(lm_dt[(lm_dt.gene_name==row.gene_name)]['deeprvat'], lm_dt[(lm_dt.gene_name==row.gene_name)][f'{row.trait}_common_resid'])
        
    slopes.append(slope)
    slope_SEs.append(se)
    pvals.append(p)


plt_genes['slope'] = slopes
plt_genes['slope_SE'] = slope_SEs
plt_genes['pval'] = pvals
plt_genes

In [ ]:
# plt_genes['p_label'] = plt_genes.nom_pval.apply(lambda x: f"BD p-val: {x:.5f}")
plt_genes['slope_label'] = "evaluation slope: " + round(plt_genes["slope"],2).astype(str) + "±" + round(1.96*plt_genes["slope_SE"],2).astype(str)
plt_genes['p_label'] = plt_genes.pval.apply(lambda x: f"slope p-value: {x:.5f}")
plt_genes['FuncRVP_estimate'] = plt_genes.beta.apply(lambda x: f"FuncRVP estimate: {x:.2f}")

plt_genes['slope_signif'] = np.where(np.abs(plt_genes["slope"]) > 1.96*plt_genes["slope_SE"], True, False)
plt_genes

In [ ]:
import patchworklib as pw

def make_plot(gene_name, trait):
    al_plot = (pn.ggplot(plot_dt[plot_dt.gene_name == gene_name], 
                     pn.aes(x='deeprvat', y=f'{trait}_common_resid')) 
           + pn.geom_point(pn.aes(color='plof'), alpha=0.5, size=1)
           + pn.geom_hline(yintercept = 0, linetype='dashed')
           # + pn.geom_smooth(method='lm')
           + pn.stat_smooth(method='lm')
           + pn.scale_color_manual(name='pLoF', values = ["darkgrey", "red"])
           + pn.ylab(f"{trait}\n(adjusted)")
           + pn.xlab("DeepRVAT score")
           # + pn.geom_text(pn.aes(x=np.Inf, y=np.Inf, label="p_label"), data=plt_genes[(plt_genes.trait == trait) & (plt_genes.gene_name == gene_name)], size=13)
           # + pn.geom_text(pn.aes(x=np.Inf, y=np.Inf, label="beta_label"), data=plt_genes[(plt_genes.trait == trait) & (plt_genes.gene_name == gene_name)], size=13)
           # + pn.geom_text(pn.aes(x=0.755, y=3.5, label="p_label"), data=plt_genes[(plt_genes.trait == trait) & (plt_genes.gene_name == gene_name)], size=13)
           + pn.geom_text(pn.aes(x=1, y=4, label="FuncRVP_estimate"), data=plt_genes[(plt_genes.trait == trait) & (plt_genes.gene_name == gene_name)], size=14, ha = "right")
           + pn.geom_text(pn.aes(x=1, y=3.7, label="slope_label"), data=plt_genes[(plt_genes.trait == trait) & (plt_genes.gene_name == gene_name)], size=14, ha = "right")
           # + pn.geom_text(pn.aes(x=1, y=3.4, label="p_label"), data=plt_genes[(plt_genes.trait == trait) & (plt_genes.gene_name == gene_name)], size=14, ha = "right")
           + pn.ggtitle(f"{gene_name}")
           + pn.ylim([-4,4])
           + pn.theme_classic()           
           + pn.theme(text=pn.element_text(size=15),
                      plot_title = pn.element_text(hjust = 0.5),
                      legend_position='bottom'))

    return al_plot

In [ ]:
%%time

plot_vec = []
# for tr in trait_gene_dict:
for tr in ['Lymphocyte_percentage', 'Platelet_crit']:
    plot_vec.append(pw.load_ggplot(make_plot(gene_name=trait_gene_dict[tr], trait=tr), figsize=(3,6)))


In [ ]:
%%time

# g1234 = (plot_vec[0]|plot_vec[1])/(plot_vec[2]|plot_vec[3])
# g1234 = plot_vec[0]|plot_vec[1]|plot_vec[2]|plot_vec[3]
g1234 = plot_vec[0]|plot_vec[1]
g1234

In [ ]:
g1234.savefig('/s/project/geno2pheno/figures/figure_6_new.png', dpi=96)

In [ ]:
gene_id = "ENSG00000139725"
phenocode = "30090"
trait = "Platelet_crit"
com_covs = get_common_variant_covariates(gene_id, phenocode=phenocode)
# com_covs

lm_dt = plot_dt.dropna()

a = pd.merge(com_covs, lm_dt[(lm_dt.gene_id==gene_id)][[f'{trait}_common_resid', 'individual']], on='individual')
a = a.set_index('individual')

lm = LinearRegression().fit(a.drop(columns=f'{trait}_common_resid'), a[f'{trait}_common_resid'])
prediction = lm.predict(a.drop(columns=f'{trait}_common_resid'))
residual = a[f'{trait}_common_resid'] - prediction
residual = residual.rename(f'{trait}_residual')
residual

In [ ]:
a[f'{trait}_common_resid'].hist(bins=50)

In [ ]:
residual.hist(bins=50)